In [3]:
%load_ext autoreload
%autoreload 2

import spacy, random, math, time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.datasets import TranslationDataset, Multi30k, IWSLT
from torchtext.data import Field, BucketIterator, RawField, Dataset

from models.gcn import GCNLayer
from tqdm import tqdm

#### Multi30k
---

In [4]:
SEED = 11747
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text, reverse=False):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    tokens = [tok.text for tok in spacy_de.tokenizer(text)]
    return tokens[::-1] if reverse else tokens
    
def tokenize_en(text, reverse=False):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    tokens = [tok.text for tok in spacy_en.tokenizer(text)]
    return tokens[::-1] if reverse else tokens
    
def batch_graph(grhs):
    """ batch a list of graphs
    @param grhs: list(tensor,...) 
    """
    b = len(grhs)  # batch size
    graph_dims = [len(g) for g in grhs]
    s = max(graph_dims)  # max seq length
    
    G = torch.zeros([b, s, s])
    for i, g in enumerate(grhs):
        s_ = graph_dims[i]
        G[i,:s_,:s_] = g
    return G

In [4]:
REVERSE = True
SRC = Field(tokenize = lambda text: tokenize_de(text, REVERSE), 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TGT = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
GRH = RawField(postprocessing=batch_graph)
data_fields = [('src', SRC), ('trg', TGT), ('grh', GRH)]

train_data = Dataset(torch.load("data/Multi30k/train_data.pt"), data_fields)
valid_data = Dataset(torch.load("data/Multi30k/valid_data.pt"), data_fields)
test_data = Dataset(torch.load("data/Multi30k/test_data.pt"), data_fields)

In [5]:
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=False,
    device = device)

In [6]:
SRC.build_vocab(train_data, min_freq = 2)
TGT.build_vocab(train_data, min_freq = 2)

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TGT.vocab)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [7]:
from collections import Counter

def get_sentence_lengths(dataset):
    src_counter = Counter()
    tgt_counter = Counter()
    for exp in dataset:
        src_counter[len(exp.src)] += 1
        tgt_counter[len(exp.trg)] += 1
    return src_counter, tgt_counter

def counter2array(counter):
    result = []
    for k in counter:
        result.extend([k for _ in range(counter[k])])
    return np.array(result)

In [8]:
src_c, tgt_c = get_sentence_lengths(train_data)
src_lengths = counter2array(src_c)
tgt_lengths = counter2array(tgt_c)

print("maximum src, tgt sent lengths: ")
np.quantile(src_lengths, 1), np.quantile(tgt_lengths, 1)

maximum src, tgt sent lengths: 


(44, 41)

#### Experiment with just GRU
---

In [9]:
class GRUEncoder(nn.Module):
    def __init__(self, ninp, nembed, enc_nhid, dec_nhid, nlayers, dropout):
        super(GRUEncoder, self).__init__()
        self.enc_nhid = enc_nhid
        self.dec_nhid = dec_nhid
        self.nlayers = nlayers
        self.embedding = nn.Embedding(ninp, nembed)
        self.rnn = nn.GRU(nembed, enc_nhid, nlayers, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(enc_nhid*2, dec_nhid)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src: (src len, b)
        s, b = src.shape
        embedded = self.dropout(self.embedding(src))
        out, hidden = self.rnn(embedded)
        hidden = hidden.transpose(1,0).reshape(b, self.nlayers, -1).transpose(1,0)
        hidden = torch.tanh(self.fc(hidden))
        return out, hidden

In [10]:
class GRUDecoder(nn.Module):
    def __init__(self, nout, nembed, enc_nhid, dec_nhid, nlayers, dropout):
        super(GRUDecoder, self).__init__()
        self.nout = nout
        self.enc_nhid = enc_nhid
        self.dec_nhid = dec_nhid
        self.nlayers = nlayers
        self.embedding = nn.Embedding(nout, nembed)
        self.rnn = nn.GRU(nembed, dec_nhid, nlayers, bidirectional=False, dropout=dropout)
        self.fc_out = nn.Linear(dec_nhid, nout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, hidden):
        x = x.unsqueeze(0)
        embedded = self.dropout(self.embedding(x))
        out, hidden = self.rnn(embedded, hidden)
        pred = self.fc_out(out.squeeze(0))
        return pred, hidden

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        # src: (src_len, b)
        # tgt: (tgt_len, b)
        tgt_len, b = tgt.shape
        tgt_vocab_size = self.decoder.nout
        
        # tensor to store decoder outputs
        outs = torch.zeros(tgt_len, b, tgt_vocab_size).to(self.device)
        
        enc_out, hidden = self.encoder(src)
        x = tgt[0]
        for t in range(1, tgt_len):
            out, hidden = self.decoder(x, hidden)
            outs[t] = out
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = out.argmax(1)
            x = tgt[t] if teacher_force else top1
            
        return outs

In [12]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)
ENC_EMB_DIM = 250
DEC_EMB_DIM = 250
ENC_HID_DIM = 500
DEC_HID_DIM = 500
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
NLAYERS = 2

enc = GRUEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, ENC_DROPOUT)
dec = GRUDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

In [13]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): GRUEncoder(
    (embedding): Embedding(7855, 250)
    (rnn): GRU(250, 500, num_layers=2, dropout=0.5, bidirectional=True)
    (fc): Linear(in_features=1000, out_features=500, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): GRUDecoder(
    (embedding): Embedding(5893, 250)
    (rnn): GRU(250, 500, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=500, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,282,893 trainable parameters


In [15]:
optimizer = optim.Adam(model.parameters())
TGT_PAD_IDX = TGT.vocab.stoi[TGT.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

In [16]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    train_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        tgt = batch.trg
        optimizer.zero_grad()
        out = model(src, tgt)
        out_dim = out.shape[-1]
        out = out[1:].view(-1, out_dim)
        tgt = tgt[1:].view(-1)
        
        loss = criterion(out, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
    return train_loss/len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            tgt = batch.trg
            out = model(src, tgt, 0) # no teacher forcing here
            out_dim = out.shape[-1]
            out = out[1:].view(-1, out_dim)
            tgt = tgt[1:].view(-1)
            loss += criterion(out, tgt).item()
    return loss/len(iterator)


def epoch_time(start, end):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [17]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 30s
	Train Loss: 4.789 | Train PPL: 120.187
	 Val. Loss: 4.878 |  Val. PPL: 131.371
Epoch: 02 | Time: 0m 30s
	Train Loss: 3.957 | Train PPL:  52.298
	 Val. Loss: 4.547 |  Val. PPL:  94.388
Epoch: 03 | Time: 0m 30s
	Train Loss: 3.582 | Train PPL:  35.931
	 Val. Loss: 4.202 |  Val. PPL:  66.814
Epoch: 04 | Time: 0m 31s
	Train Loss: 3.327 | Train PPL:  27.867
	 Val. Loss: 4.076 |  Val. PPL:  58.906
Epoch: 05 | Time: 0m 30s
	Train Loss: 3.124 | Train PPL:  22.729
	 Val. Loss: 3.854 |  Val. PPL:  47.191
Epoch: 06 | Time: 0m 31s
	Train Loss: 2.972 | Train PPL:  19.537
	 Val. Loss: 3.764 |  Val. PPL:  43.124
Epoch: 07 | Time: 0m 31s
	Train Loss: 2.843 | Train PPL:  17.172
	 Val. Loss: 3.759 |  Val. PPL:  42.924
Epoch: 08 | Time: 0m 31s
	Train Loss: 2.710 | Train PPL:  15.030
	 Val. Loss: 3.757 |  Val. PPL:  42.835
Epoch: 09 | Time: 0m 31s
	Train Loss: 2.582 | Train PPL:  13.219
	 Val. Loss: 3.689 |  Val. PPL:  40.015
Epoch: 10 | Time: 0m 31s
	Train Loss: 2.498 | Train PPL

#### GCN Incorporated
---

In [18]:
class GCNEncoder(nn.Module):
    def __init__(self, ninp, nembed, nhid, nlayers, dropout):
        super(GCNEncoder, self).__init__()
        self.nhid = nhid
        self.nlayers = nlayers
        self.embedding = nn.Embedding(ninp, nembed)
        assert(nlayers > 0)
        layers = [GCNLayer(nembed, nhid)] + [GCNLayer(nhid, nhid) for _ in range(nlayers-1)]
        self.layers = nn.ModuleList(layers)
        self.linear = nn.Linear(2*nhid, nhid)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, A):
        """
        x: (seq len, b)
        A: (b, seq len, seq len)
        """
        x = x.t()
        b = x.shape[0]
        x = self.embedding(x)  # x: (b, seq len, ninp)
        x = self.dropout(x)
        hidden = []
        for layer in self.layers:
            x = layer(x, A) 
            hidden.append(x[:,0,:])
            
        # pooling
        mean = x.mean(dim=1)
        maxm = x.max(dim=1)[0]
        x = torch.cat((mean, maxm), dim=1)
        out = self.linear(self.dropout(x))
        hidden = torch.stack(hidden)
        return out, hidden

        
        

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, grh, tgt, teacher_forcing_ratio=0.5):
        # src: (src_len, b)
        # tgt: (tgt_len, b)
        tgt_len, b = tgt.shape
        tgt_vocab_size = self.decoder.nout
        
        # tensor to store decoder outputs
        outs = torch.zeros(tgt_len, b, tgt_vocab_size).to(self.device)
        
        enc_out, hidden = self.encoder(src, grh)
        x = tgt[0]
        for t in range(1, tgt_len):
            out, hidden = self.decoder(x, hidden)
            outs[t] = out
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = out.argmax(1)
            x = tgt[t] if teacher_force else top1
            
        return outs

In [19]:
REVERSE = False
SRC = Field(tokenize = lambda text: tokenize_de(text, REVERSE), 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TGT = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
GRH = RawField(postprocessing=batch_graph)
data_fields = [('src', SRC), ('trg', TGT), ('grh', GRH)]

train_data = Dataset(torch.load("data/Multi30k/train_data.pt"), data_fields)
valid_data = Dataset(torch.load("data/Multi30k/valid_data.pt"), data_fields)
test_data = Dataset(torch.load("data/Multi30k/test_data.pt"), data_fields)

BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=False,
    device = device)

SRC.build_vocab(train_data, min_freq = 2)
TGT.build_vocab(train_data, min_freq = 2)

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TGT.vocab)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [20]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 700
DEC_HID_DIM = 700
N_LAYERS = 3
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = GCNEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = GRUDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,692,681 trainable parameters


In [22]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    train_loss = 0
    for i, batch in tqdm(enumerate(iterator)):
        src = batch.src.to(device)
        tgt = batch.trg.to(device)
        grh = batch.grh.to(device)
        optimizer.zero_grad()
        out = model(src, grh, tgt)
        out_dim = out.shape[-1]
        out = out[1:].view(-1, out_dim)
        tgt = tgt[1:].view(-1)
        
        loss = criterion(out, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
    return train_loss/len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    loss = 0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(iterator)):
            src = batch.src.to(device)
            tgt = batch.trg.to(device)
            grh = batch.grh.to(device)
            out = model(src, grh, tgt, 0) # no teacher forcing here
            out_dim = out.shape[-1]
            out = out[1:].view(-1, out_dim)
            tgt = tgt[1:].view(-1)
            loss += criterion(out, tgt).item()
    return loss/len(iterator)


def epoch_time(start, end):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10
CLIP = 1

optimizer = optim.Adam(model.parameters())
TGT_PAD_IDX = TGT.vocab.stoi[TGT.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

227it [00:35,  6.47it/s]
8it [00:00, 23.45it/s]
0it [00:00, ?it/s]

Epoch: 01 | Time: 0m 35s
	Train Loss: 4.834 | Train PPL: 125.671
	 Val. Loss: 4.738 |  Val. PPL: 114.180


227it [00:35,  6.37it/s]
8it [00:00, 22.95it/s]
1it [00:00,  5.64it/s]

Epoch: 02 | Time: 0m 35s
	Train Loss: 4.186 | Train PPL:  65.776
	 Val. Loss: 4.625 |  Val. PPL: 102.045


227it [00:35,  6.44it/s]
8it [00:00, 23.33it/s]
1it [00:00,  6.05it/s]

...

Epoch: 08 | Time: 0m 35s
	Train Loss: 3.494 | Train PPL:  32.931
	 Val. Loss: 4.416 |  Val. PPL:  82.803


227it [00:35,  6.40it/s]
8it [00:00, 23.11it/s]
1it [00:00,  6.18it/s]

Epoch: 09 | Time: 0m 35s
	Train Loss: 3.460 | Train PPL:  31.820
	 Val. Loss: 4.460 |  Val. PPL:  86.466


227it [00:35,  6.43it/s]
8it [00:00, 23.65it/s]

Epoch: 10 | Time: 0m 35s
	Train Loss: 3.379 | Train PPL:  29.339
	 Val. Loss: 4.418 |  Val. PPL:  82.900


#### GCN and GRU Combined

In [24]:
class GCNGRUEncoder(nn.Module):

    def __init__(self, ninp, nembed, enc_nhid, dec_nhid, nlayers, dropout, device):
        super(GCNGRUEncoder, self).__init__()
        self.enc_nhid = enc_nhid
        self.dec_nhid = dec_nhid
        self.nlayers = nlayers
        self.embedding = nn.Embedding(ninp, nembed)
        self.device = device
        rnns = [nn.GRU(nembed, enc_nhid, 1, bidirectional=True)] + \
               [nn.GRU(enc_nhid*2, enc_nhid, 1, bidirectional=True) 
                for _ in range(nlayers-1)]
        self.rnns = nn.ModuleList(rnns)
        layers = [GCNLayer(nembed, enc_nhid)] + [GCNLayer(enc_nhid, enc_nhid) 
                                                   for _ in range(nlayers-1)]
        self.gcns = nn.ModuleList(layers)
        self.proj = nn.Linear(enc_nhid*2, enc_nhid)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, A):
        """
        x: (seq len, b)
        A: (b, seq len, seq len)
        """
        s, b = x.shape
        embedded = self.dropout(self.embedding(x))
        hiddens = []
        hidden = torch.zeros(2, b, self.enc_nhid).to(self.device)
        gcn_out, gru_out = embedded.transpose(1,0), embedded
        for i in range(self.nlayers):
            gcn_out = self.gcns[i](gcn_out, A)
            gru_out, hidden = self.rnns[i](gru_out, hidden)
            gru_out = self.dropout(gru_out)            
            gru_out = gcn_out.repeat(1,1,2).transpose(0,1) + gru_out
            gcn_out = self.proj(gru_out.transpose(0,1))
            hidden += gcn_out.max(1)[0].repeat(2,1,1)
            hiddens.append(hidden.sum(dim=0))
        hidden = torch.stack(hiddens)
        return gru_out, hidden

In [25]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
N_LAYERS = 3
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

enc = GCNGRUEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, ENC_DROPOUT, device)
dec = GRUDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

In [26]:
REVERSE = False
SRC = Field(tokenize = lambda text: tokenize_de(text, REVERSE), 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TGT = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
GRH = RawField(postprocessing=batch_graph)
data_fields = [('src', SRC), ('trg', TGT), ('grh', GRH)]

train_data = Dataset(torch.load("data/Multi30k/train_data.pt"), data_fields)
valid_data = Dataset(torch.load("data/Multi30k/valid_data.pt"), data_fields)
test_data = Dataset(torch.load("data/Multi30k/test_data.pt"), data_fields)

BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=False,
    device = device)

SRC.build_vocab(train_data, min_freq = 2)
TGT.build_vocab(train_data, min_freq = 2)

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TGT.vocab)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [27]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 23,873,797 trainable parameters


In [28]:
N_EPOCHS = 10
CLIP = 1

optimizer = optim.Adam(model.parameters())
TGT_PAD_IDX = TGT.vocab.stoi[TGT.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

227it [00:43,  5.28it/s]
8it [00:00, 23.54it/s]
1it [00:00,  5.29it/s]

Epoch: 01 | Time: 0m 43s
	Train Loss: 4.641 | Train PPL: 103.656
	 Val. Loss: 4.749 |  Val. PPL: 115.418


227it [00:43,  5.27it/s]
8it [00:00, 23.14it/s]
0it [00:00, ?it/s]

Epoch: 02 | Time: 0m 43s
	Train Loss: 3.692 | Train PPL:  40.125
	 Val. Loss: 4.222 |  Val. PPL:  68.146


227it [00:42,  5.30it/s]
8it [00:00, 23.81it/s]
0it [00:00, ?it/s]

...

Epoch: 08 | Time: 0m 43s
	Train Loss: 2.460 | Train PPL:  11.703
	 Val. Loss: 3.926 |  Val. PPL:  50.701


227it [00:42,  5.29it/s]
8it [00:00, 23.61it/s]
0it [00:00, ?it/s]

Epoch: 09 | Time: 0m 43s
	Train Loss: 2.375 | Train PPL:  10.753
	 Val. Loss: 3.843 |  Val. PPL:  46.683


227it [00:43,  5.27it/s]
8it [00:00, 23.85it/s]

Epoch: 10 | Time: 0m 43s
	Train Loss: 2.295 | Train PPL:   9.921
	 Val. Loss: 3.862 |  Val. PPL:  47.574


#### GCN Attention Encoder

In [63]:
from models.gru_attn import Attention, GRUDecoder

class GCNEncoder(nn.Module):
    def __init__(self, ninp, nembed, enc_nhid, dec_nhid, nlayers, dropout):
        super(GCNEncoder, self).__init__()
        self.enc_nhid = enc_nhid
        self.dec_nhid = dec_nhid
        self.nlayers = nlayers
        self.embedding = nn.Embedding(ninp, nembed)
        assert(nlayers > 0)
        layers = [GCNLayer(nembed, enc_nhid)] + \
                 [GCNLayer(enc_nhid, enc_nhid) for _ in range(nlayers-1)]
        self.layers = nn.ModuleList(layers)
        self.out_linear = nn.Linear(enc_nhid, 2*enc_nhid)
        self.hid_linear = nn.Linear(enc_nhid, dec_nhid)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, A):
        """
        x: (seq len, b)
        A: (b, seq len, seq len)
        """
        x = x.t()
        b = x.shape[0]
        x = self.embedding(x)  # x: (b, seq len, ninp)
        x = self.dropout(x)
        hidden = []
        for layer in self.layers:
            x = layer(x, A) 
            hidden.append(x[:,0,:])
            
        # pooling
        out = self.out_linear(self.dropout(x)).transpose(1,0)
        hidden = torch.stack(hidden)
        hidden = self.hid_linear(hidden)
        return out, hidden
    
    
class GCNGRUEncoder(nn.Module):
    def __init__(self, ninp, nembed, enc_nhid, dec_nhid, nlayers, dropout, device=torch.device('cpu')):
        super(GCNGRUEncoder, self).__init__()
        self.enc_nhid = enc_nhid
        self.dec_nhid = dec_nhid
        self.nlayers = nlayers
        assert(nlayers > 0)
        self.embedding = nn.Embedding(ninp, nembed)
        grus = [nn.GRU(nembed, enc_nhid, 1, bidirectional=True)] + \
               [nn.GRU(enc_nhid*2, enc_nhid, 1, bidirectional=True)
                for _ in range(nlayers-1)] 
        gcns = [GCNLayer(nembed, enc_nhid)] + \
               [GCNLayer(enc_nhid, enc_nhid) for _ in range(nlayers-1)]
        self.grus = nn.ModuleList(grus)
        self.gcns = nn.ModuleList(gcns)
        self.proj = nn.Linear(enc_nhid*2, enc_nhid)
        self.dropout = nn.Dropout(dropout)
        self.device = device
        
    def forward(self, x, A):
        s, b = x.shape
        embedded = self.dropout(self.embedding(x))
        hiddens = []
        hidden = torch.zeros(2, b, self.enc_nhid).to(self.device)
        gcn_out, gru_out = embedded.transpose(1,0), embedded
        for i in range(self.nlayers):
            gcn_out = self.gcns[i](gcn_out, A)
            gru_out, hidden = self.grus[i](gru_out, hidden)
            gru_out = self.dropout(gru_out)
            gru_out = gcn_out.repeat(1,1,2).transpose(0,1) + gru_out
            gcn_out = self.proj(gru_out.transpose(0,1))
            hidden += gcn_out.max(1)[0].repeat(2,1,1)
            hiddens.append(hidden.sum(dim=0))
        hidden = torch.stack(hiddens)
        return gru_out, hidden


In [64]:
class GCN2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(GCN2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, grh, tgt, teacher_forcing_ratio=0.5):
        # src: (src_len, b)
        # tgt: (tgt_len, b)
        tgt_len, b = tgt.shape
        tgt_vocab_size = self.decoder.nout
        
        # tensor to store decoder outputs
        outs = torch.zeros(tgt_len, b, tgt_vocab_size).to(self.device)
        
        enc_out, hidden = self.encoder(src, grh)
        x = tgt[0]
        attns = []
        for t in range(1, tgt_len):
            out, hidden, attn = self.decoder(x, hidden, enc_out)
            attns.append(attn)
            outs[t] = out
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = out.argmax(1)
            x = tgt[t] if teacher_force else top1
        attns = torch.stack(attns, dim=1).squeeze()
        return outs, attns

In [65]:
REVERSE = False
SRC = Field(tokenize = lambda text: tokenize_de(text, REVERSE), 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TGT = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
GRH = RawField(postprocessing=batch_graph)
data_fields = [('src', SRC), ('trg', TGT), ('grh', GRH)]

train_data = Dataset(torch.load("data/Multi30k/train_data.pt"), data_fields)
valid_data = Dataset(torch.load("data/Multi30k/valid_data.pt"), data_fields)
test_data = Dataset(torch.load("data/Multi30k/test_data.pt"), data_fields)

BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.src),
    sort_within_batch=False,
    device = device)

SRC.build_vocab(train_data, min_freq = 2)
TGT.build_vocab(train_data, min_freq = 2)

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TGT.vocab)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [66]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TGT.vocab)
ENC_EMB_DIM = 250
DEC_EMB_DIM = 250
ENC_HID_DIM = 500
DEC_HID_DIM = 500
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
NLAYERS = 2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
# enc = GCNEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, ENC_DROPOUT)
enc = GCNGRUEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, ENC_DROPOUT, device)
dec = GRUDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, NLAYERS, DEC_DROPOUT, attn)
model = GCN2Seq(enc, dec, device).to(device)

In [67]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 26,276,143 trainable parameters


In [68]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    train_loss = 0
    for i, batch in tqdm(enumerate(iterator)):
        src = batch.src.to(device)
        tgt = batch.trg.to(device)
        grh = batch.grh.to(device)
        optimizer.zero_grad()
        out, attns = model(src, grh, tgt)
        out_dim = out.shape[-1]
        out = out[1:].view(-1, out_dim)
        tgt = tgt[1:].view(-1)
        
        loss = criterion(out, tgt)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
    return train_loss/len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    loss = 0
    with torch.no_grad():
        for i, batch in tqdm(enumerate(iterator)):
            src = batch.src.to(device)
            tgt = batch.trg.to(device)
            grh = batch.grh.to(device)
            out, attns = model(src, grh, tgt, 0) # no teacher forcing here
            out_dim = out.shape[-1]
            out = out[1:].view(-1, out_dim)
            tgt = tgt[1:].view(-1)
            loss += criterion(out, tgt).item()
    return loss/len(iterator)

def epoch_time(start, end):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [69]:
N_EPOCHS = 10
CLIP = 1

optimizer = optim.Adam(model.parameters())
TGT_PAD_IDX = TGT.vocab.stoi[TGT.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

0it [00:00, ?it/s]

KeyboardInterrupt: 

#### GCNGRU Attention Encoder

#### Transformer with Attention

#### Baby test for GCN Encoder

In [12]:
for b in train_iterator:
    x = b.src.to(device)
    A = b.grh.to(device)
    break

out, hidden = enc(x, A)

AttributeError: 'GCNGRUEncoder' object has no attribute 'embedding'

In [13]:
hidden.shape

NameError: name 'hidden' is not defined